# Assignment 2




In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import metrics


import os
import random

SEED = 42
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)

## Task 1 (20 points)

Under what assumption Decision Tree performes worse than Linear Regression?  
Create a dataset, choose train/test split, and make experiment to support the claim.  
Use `sklearn.linear_model.LinearRegression`, `sklearn.tree.DecisionTreeRegressor`, `sklearn.datasets.make_regression`  
Use MSE loss.

## Task 2 (20 points)
Make dataset preprocessing as described below 

In [ ]:
from sklearn import preprocessing
from tqdm import tqdm
from sklearn import base

In [ ]:
df = pd.read_csv('data.csv')
del df['id']
df.head()

In [ ]:
df['month'] = df['month'].map({'January':1, 'February':2, 'March': 3, 'April': 4, 
                               'May':5, 'June':6, 'July':7, 'August':8})

In [ ]:
df['credit_history_age'].head()

In [ ]:
# TODO: convert column into number of months (float). Fill missing data with 0

df['credit_history_age'] = ...

In [ ]:
df['type_of_loan'].head()

In [ ]:
# TODO type_of_loan is a combination of flags
# for each type of loan create a seperate colum with binary values.
# fill missing data with 0

loan_types = ..
for col in loan_types:
    col_name = col.replace(' ', '_')
    df[col_name] = ...
    
del df['type_of_loan']

In [ ]:
nan_columns = [
    'monthly_inhand_salary',
    'num_of_delayed_payment',
    'num_credit_inquiries',
    'credit_history_age',
    'amount_invested_monthly',
    'monthly_balance'
]

# TODO replace missing values in the columns with an average over values with the same client_id 
# If all values within the same client_id are missing, replace them with an average over entire column

for col in tqdm(nan_columns): 
    df[col] = ...

In [ ]:
ctg_columns = [
            'customer_id', 'name', 'credit_score', 'payment_behaviour', 'occupation', 'ssn', 
            'payment_of_min_amount', 'credit_mix', 
        ]

# TODO encode column values with integers. Use sklearn.preprocessing.LabelEncoder

for col in ctg_columns:
    df[col] = ...

In [ ]:
def fix(x):
    if isinstance(x, str):
        x = x.replace('_', '')
        if len(x) > 0:
            return float(x)
        else:
            return 0
    else:
        return float(x)
    
for col in df.columns:
    df[col] = df[col].apply(fix)

In [ ]:
if np.all(pd.isnull(df).sum() == 0) and np.isnan(df.values.astype(float)).sum():
    print(success)

In [ ]:
df_test = df[df['month'] == 8].reset_index(drop=True)
df_train = df[df['month'] != 8].reset_index(drop=True)
len(df_train)

In [ ]:
X_test, y_test = df_test.drop(columns='credit_score').values, df_test['credit_score'].values
X_train, y_train = df_train.drop(columns='credit_score').values, df_train['credit_score'].values

## Task 3 (20 points)
Fit Random Forest Classifier on the dataset.  
Implement grid search with some cross-validation schema to select hyperparams for Random Forest.  
Surpass baseline F1_macro = 0.72 on the test subset  
Tune only most sensitive hyperparams, consider your computational budget.  

Note: you cannot use sklearn.model_selection.GridSearchCV

In [ ]:
from sklearn import model_selection
from sklearn import ensemble

In [ ]:
hyp1_space = ...
...
hypk_space = ...

records = []


for train, valid in split(X_train, y_train, ...):
    
    for hyp1_value in hyp1_space:
        ...
        for hypk_value in hypk_space:
            ...
            model = ensemble.RandomForestClassifier(...)
            model.fit(...)
            y_pred = model.predict(X_test)
            score = metrics.f1_score(y_valid, y_pred, average='macro')
            records.append({'hyp1': hyp1, ..., 'hypk': hypk,  'score': score})

In [ ]:
records = pd.DataFrame(records)
records.groupby(['hyp1', ... ,'hypk']).mean()

In [ ]:
best_params = ...

# don't forget to retrain model on the whole train dataset

y_pred = model.predict(...)
score = metrics.f1_score(y_test, y_pred, average='macro')

if score >= 0.72:
    print('success')

## Task 4 (20 points)

Fit XGBoost classifier on the dataset.  
Implement grid search with hold-out validation schema to select hyperparams.  
Surpass baseline F1_macro = 0.73 on the test subset  
Tune only most sensitive hyperparams, consider your computational budget.  

Note: use additional arguments (eval_metric, eval_set) in XGBoost classifier to make early stopping.

In [ ]:
import xgboost as xgb

In [ ]:
hyp1_space = ...
...
hypk_space = ...

records = []

train, valid = split(X_train, y_train, ...):
    
for hyp1_value in hyp1_space:
    ...
    for hypk_value in hypk_space:
        ...
        model = xgb.XGBClassifier(...)
        model.fit(...)
        y_pred = model.predict(...)
        score = metrics.f1_score(y_valid, y_pred, average='macro')
        records.append({'hyp1': hyp1, ..., 'hypk': hypk,  'score': score})

In [ ]:
records = pd.DataFrame(records)
records.loc[records['score'].argmax()]

In [ ]:
best_params = ...

# don't forget to retrain model on the whole train dataset

y_pred = model.predict(X_test)
score = metrics.f1_score(y_test, y_pred, average='macro')

if score >= 0.73:
    print('success')

## Task 5 (20 points)

How feature importance is connected to the underlying model?   
Select top 10 most important features from Logistic Regression and Random Forest.   
On these selected features fit logistic regression with L2 norm regularization and some fixed regularization coefficient.  
Which selected features are better in terms of F1_macro score?
